In [1]:
import json
import pandas as pd
import nltk
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import glob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import json
import warnings
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import reuters
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk.data
import math
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd
import scipy as sc

import os
import json
import warnings

from sklearn.metrics.pairwise import cosine_similarity
DEBUG = False
SUMMARY_LENGTH = 12  # number of sentences in final summary
stop_words = stopwords.words('english')
ideal_sent_length = 20.0
stemmer = SnowballStemmer("english")
from sklearn.metrics.pairwise import cosine_similarity
import re
from urllib.request import urlopen
#import gensim 
import numpy as np 
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import re
import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim

import spacy
import joblib


from IPython.display import HTML, display
from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

from tqdm import tqdm

import seaborn as sb
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import nltk
from nltk.corpus import wordnet as wn

In [2]:
path = r"C:\Users\ThinkPad\OneDrive\Bureau\CORD-19-research-challenge"
all_json = glob.glob(f'{path}/**/*.json', recursive=True)
len(all_json)
metadata_path = f'{path}/metadata.csv'
# load metadata
metadata = pd.read_csv(metadata_path)
metadata.head()
metadata.sha.isnull().sum()

15758

In [3]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            dic={'title':[]} 
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.metadataa = []
            self.metadata = content['metadata']
            meta=self.metadata
            self.abstract = []
            self.body_text = []
            
            
           
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
                
           
                tit=meta['title']
                dic['title'].append(tit)
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            self.metadataa = tit
    def __repr__(self):
        return f'{self.metadataa}...'
first_row = FileReader(all_json[0])
print(first_row)

The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus. 2 3...


In [4]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [],'title':[]}
for idx, entry in enumerate(all_json[0:1000]):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['title'].append(content.metadataa)
    
    
papers = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text','title'])
papers.head()

Processing index: 0 of 29315


,paper_id,abstract,body_text,title
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...",The RNA pseudoknots in foot-and-mouth disease ...
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,Healthcare-resource-adjusted vulnerabilities t...
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...","Real-time, MinION-based, amplicon sequencing f..."
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...,A Combined Evidence Approach to Prioritize Nip...
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p...",Assessing spread risk of Wuhan novel coronavir...


In [5]:
# any true nulls in the data?
papers.isnull().sum(axis=0)
# a few dupes - get rid of those
papers.drop_duplicates('body_text', inplace=True)
papers.shape

(1000, 4)

In [6]:
full_df = papers\
    .merge(metadata.rename(columns={'sha':'paper_id'}).drop(['abstract','title'], axis=1), 
           on='paper_id', how='left')

full_df.head()

,paper_id,abstract,body_text,title,source_x,doi,pmcid,pubmed_id,license,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...",The RNA pseudoknots in foot-and-mouth disease ...,biorxiv,10.1101/2020.01.10.901801,NaN,NaN,biorxiv,2020-01-11,"Ward, J. C. J.; Lasecka-Dykes, L.; Neil, C.; A...",NaN,NaN,NaN,True,biorxiv_medrxiv
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,Healthcare-resource-adjusted vulnerabilities t...,medrxiv,10.1101/2020.02.11.20022111,NaN,NaN,medrvix,2020-02-12,Hanchu Zhou; Jianan Yang; Kaichen Tang; Qingpe...,NaN,NaN,NaN,True,biorxiv_medrxiv
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...","Real-time, MinION-based, amplicon sequencing f...",biorxiv,10.1101/634600,NaN,NaN,biorxiv,2019-05-10,"Butt, S. L.; Erwood, E. C.; Zhang, J.; Sellers...",NaN,NaN,NaN,True,biorxiv_medrxiv
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...,A Combined Evidence Approach to Prioritize Nip...,biorxiv,10.1101/2020.03.12.977918,NaN,NaN,biorxiv,2020-03-12,Nishi Kumari; Ayush Upadhyay; Kishan Kalia; Ra...,NaN,NaN,NaN,True,biorxiv_medrxiv
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p...",Assessing spread risk of Wuhan novel coronavir...,medrxiv,10.1101/2020.02.04.20020479,NaN,NaN,medrvix,2020-02-05,Shengjie Lai; Isaac Bogoch; Nick Ruktanonchai;...,NaN,NaN,NaN,True,biorxiv_medrxiv


In [7]:
covid_csv = pd.read_csv(r'C:\Users\ThinkPad\OneDrive\Bureau\CORD-19-research-challenge\metadata.csv')
covid_csv.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license


In [8]:
full_df.drop_duplicates('body_text', inplace=True)
print(f"{full_df.shape[0]} rows in the output dataframe")

full_df.to_csv('covid_corpus_33k.csv', index=False)

1000 rows in the output dataframe


In [9]:
pre1 = covid_csv.iloc[:,0:15].copy()

In [10]:
pre = pd.DataFrame()
pre['title'] = pd.concat([pre1['title'], papers['title']], ignore_index=True)
pre['abstract'] = pd.concat([pre1['abstract'], papers['abstract']], ignore_index=True)
pre['body_text'] = pd.concat([pre1['doi'], papers['body_text']], ignore_index=True)

In [11]:
summ = pd.DataFrame()
summ['title'] =papers['title']
summ['body_text'] = papers['body_text']
summ.reset_index(drop=True, inplace=True)

In [12]:
summ.head(5)


,title,body_text
0,The RNA pseudoknots in foot-and-mouth disease ...,"VP3, and VP0 (which is further processed to VP..."
1,Healthcare-resource-adjusted vulnerabilities t...,The 2019-nCoV epidemic has spread across China...
2,"Real-time, MinION-based, amplicon sequencing f...","Infectious bronchitis (IB), which is caused by..."
3,A Combined Evidence Approach to Prioritize Nip...,Nipah is an infectious negative-sense single-s...
4,Assessing spread risk of Wuhan novel coronavir...,"In December 2019, a cluster of patients with p..."


In [13]:
summr = pd.read_csv('summr.csv')

In [14]:
summr['body_text'][0]

'VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structural protein-coding region is replaced by reporter genes, allow the study of genome 68 replication without the requirement for high containment (9, 10) ( figure 1A ).\r\nThe FMDV 5′ UTR is the largest known picornavirus UTR, comprising approximately 1300 71 nucleotides and containing several highly structured regions. The first 360 nucleotides at the 5′ 72 end are predicted to fold into a single large stem loop termed the S-fragment, followed by a The PKs were originally predicted in 1987 and consist of two to four tandem repeats of a ~48 86 nucleotide region containing a small stem loop and downstream interaction site (figure 1B) 87 (12). Due to the sequence similarity between the PKs (figure 1C), it is speculated that they 88 were formed by duplication events during viral replication, prob

In [15]:
summr['title'][0]

'The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus. 2 3'

In [16]:
SUMMARY_LENGTH = 12  # number of sentences in final summary
newcsv=pd.DataFrame()
stop_words = stopwords.words('english')
ideal_sent_length = 20.0
stemmer = SnowballStemmer("english")

In [17]:
new= pd.DataFrame()
new['title'] = pre['title']
new['body_text'] = pre['body_text']
new.head()

,title,body_text
0,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4
1,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5
2,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7
3,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9
4,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4


In [18]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [19]:
class Summarizer():
    
    
    def penn_to_wn(self,tag):
        """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
        if tag.startswith('N'):
            return 'n'
 
        if tag.startswith('V'):
            return 'v'
 
        if tag.startswith('J'):
            return 'a'
 
        if tag.startswith('R'):
            return 'r'
        return None
 

 
    def tagged_to_synset(self,word, tag):
        wn_tag = self.penn_to_wn(tag)
    
        if wn_tag is None:
            return None
 
        try:
        
            return wn.synsets(word, wn_tag)[0]
        except:
            return None
 
    def sentence_similarity(self,sentence1, sentence2):
        """ compute the sentence similarity using Wordnet """
        # Tokenize and tag
        sentence1 = pos_tag(word_tokenize(sentence1))
        
        sentence2 = pos_tag(word_tokenize(sentence2))
      
 
        # Get the synsets for the tagged words
        synsets1 = [self.tagged_to_synset(*tagged_word) for tagged_word in sentence1]
        
        synsets2 = [self.tagged_to_synset(*tagged_word) for tagged_word in sentence2]
       
 
    # Filter out the Nones
        synsets1 = [ss for ss in synsets1 if ss]
      
        synsets2 = [ss for ss in synsets2 if ss]
        
 
        score, count = 0, 0
        for synset in synsets1:
            max_sim = 0.0
            maxx=0
            for ss in synsets2:
                
                sim=wn.path_similarity(synset, ss)
                if sim is not None and sim > max_sim:
                
                       max_sim = sim
                   
           
        
            if max_sim is not None and max_sim!=0:
                score += max_sim
                count += 1
 
    # Average the values
        if count!=0:
            score /= count
            return score
   
    def __init__(self, article):
            self._articles = []
            i=1
           
            for row in article:
                if i<=1000:
                    title=row[1]
                    #print(title)
                    body=row[2].replace('\n', ' ')
                    #print(body)
                    
                    if title!=''and body!='':
                        self._articles.append((title, body))
                    i=i+1
            
            
    def valid_input(self, headline, article_text):
        return headline != '' and article_text != ''     
        
         
    def score(self, article):
        """ Assigns each sentence in the document a score"""

        headline = article[0]
        sentences = self.split_into_sentences(article[1])
        #frequency_scores = self.frequency_scores(article[1])
        #let's try a simple query
        for i, s in enumerate(sentences):
            score=self.sentence_similarity(s, 'What was the original source of COVID-19')
            if score is not None:
                self._scores[s] = score
              
    def generate_summaries(self):
        
        self.dict_ = {'title':[],'body_text': []}

        """ If article is shorter than the desired summary, just return the original articles."""
       
        for article in self._articles:
            total_num_sentences = 0
            total_num_sentences += len(self.split_into_sentences(article[1]))
        
            self._scores = Counter()
            self.score(article)
            highest_scoring = self._scores.most_common(SUMMARY_LENGTH)
            print("## Headlines: ")
            print("- " + article[0])
            print('*body**')
        # Appends highest scoring "representative" sentences, returns as a single summary paragraph.
            summr=' '.join([sent[0] for sent in highest_scoring])
            print(summr)
            self.dict_['body_text'].append(summr)
            self.dict_['title'].append(article[0])
        self.papers = pd.DataFrame(self.dict_, columns=['title','body_text'])
        return self.papers
       

    def remove_smart_quotes(self, text):
       
        """ Only concerned about smart double quotes right now. """
        #text=re.sub("([\(\[].*?[\)\]][\(\[].*?[\)\]]+)", ' ', text)
        text=re.sub("[\(\[].*?[\)\]]", '', text)
        
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', text)
            repl_url = url.group(3)
            text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, text)
        except:
            pass #there might be emails with no url in them
        #text=re.sub("[\[]()*?[\]]", "", text)#remove in-text citation
        
        text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
        text=re.sub("(\\t)", ' ', text) #remove escape charecters
        text=re.sub("(\\r)", ' ', text) 
        text=re.sub("(\\n)", ' ', text)
        text= re.sub("(\s+)",' ',text) #remove multiple spaces
        text=re.sub("doi.org doi:", ' ',text)
        text=re.sub("doi", ' ',text)
        
        return text


    def split_into_sentences(self, text):
        tok = nltk.data.load('tokenizers/punkt/english.pickle')
        sentences = tok.tokenize(self.remove_smart_quotes(text))
        sentences = [sent.replace('\n', '') for sent in sentences if len(sent) > 10]       
        for sent in sentences:
            if 'doi' in sent:
                sent.replace(sent, '')

        return sentences
    

In [20]:
import csv
f= open("summr.csv", encoding="utf-8-sig")
reader = csv.reader(f, delimiter=',')
magic = Summarizer(reader)
magic.generate_summaries()


## Headlines: 
- title
*body**

## Headlines: 
- The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus. 2 3
*body**
In addition these differences were more pronounced in more PKs is functionally competent as no differences was seen between replicons congaing a single 472 author/funder. Therefore it appears that replicons with a single PK are at a competitive 399 disadvantage compared to those with two or more. However replicons with more than a single PK 465 were found to have a competitive advantage over replicons with a single PK when sequentially 466 passaged. As near wt level of 353 replication was observed when only one PK was present all further mutagenesis was 354 performed in a C11 replicon plasmid containing only PK 1. It is unclear why some viral isolates 483 contain two three or four PKs is still unknown but this may be stochastic variation or may 484 reflect subtle effects of host rang

## Headlines: 
- Title: Viruses are a dominant driver of protein adaptation in mammals
*body**
First there may still be many undiscovered VIPs. This is most The copyright holder for this preprint is the author/funder. It is also plausible that a substantial proportion of the The copyright holder for this preprint is the author/funder. Nonetheless given that many VIPs were discovered only . Since VIPs are more 23 constrained than non-VIPs and tend to have more non-synonymous deleterious low 24 frequency variants than non-VIPs is the author/funder. The copyright holder for this preprint is the author/funder. CC-BY 4.0 International license is made available under a The copyright holder for this preprint is the author/funder. However it is unknown whether the war against viruses is fought by a 6 professional army of specifically antiviral proteins or whether it is a global war fought 7 by a broad range of VIPs. In all the 20 GO categories pN/pS is lower 3 in VIPs than in non-VIPs and the 

,title,body_text
0,title,
1,The RNA pseudoknots in foot-and-mouth disease ...,In addition these differences were more pronou...
2,Healthcare-resource-adjusted vulnerabilities t...,Xinyang Jiujiang Nanchang and Yiyang are mediu...
3,"Real-time, MinION-based, amplicon sequencing f...",One area that is problematic for a test such a...
4,A Combined Evidence Approach to Prioritize Nip...,NVIK is freely accessible at: vinodscaria.rnab...
5,Assessing spread risk of Wuhan novel coronavir...,Two Baidu datasets were used in this study. It...
6,"TWIRLS, an automated topic-wise inference meth...",It is also involved in the hydrolysis and acti...
7,Title: Viruses are a dominant driver of protei...,First there may still be many undiscovered VIP...
8,The impact of regular school closure on season...,is the author/funder. This was also found in p...
9,Carbon Nanocarriers Deliver siRNA to Intact Pl...,RNAse A was purchased from Takara Bio. Signifi...
